<a href="https://colab.research.google.com/github/WhiteChocolate0/ProgramApplication/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EX05-04 Reply Template Messages
* 依使用者輸入：confirm, buttons, carousel, imgCarousel，分別傳送確認模板、按鈕模板、輪播模板及圖片輪播模板。
* 模板上的按鈕能觸發不同類型的 Action Object。

In [ ]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 5.4 MB/s eta 0:00:00


In [3]:
# for colab
from google.colab import userdata
from flask_ngrok import run_with_ngrok
import flask_ngrok

def ngrok_start():
    ngrok_auth_token = userdata.get("NGROK_AUTHTOKEN")
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage, StickerMessage, ImageMessage, VideoMessage, LocationMessage
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get("LINE_CHANNEL_ACCESS_TOKEN"))
handler = WebhookHandler(userdata.get("LINE_CHANNEL_SECRET"))

@app.route("/callback", methods=["POST"])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return "OK"

import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel(model_name="gemini-1.0-pro")
def ask_gemini(question):
    response = model.generate_content(question)
    return response.text

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == "sticker":
            reply = StickerMessage(
                package_id="446",
                sticker_id="1989"
            )
        elif action == "image":
            reply = ImageMessage(
                original_content_url="https://cdn.pixabay.com/photo/2019/02/21/18/36/city-4011922_1280.jpg",
                preview_image_url="https://cdn.pixabay.com/photo/2019/02/21/18/36/city-4011922_1280.jpg"
            )
        elif action == "video":
            reply = VideoMessage(
                original_content_url="https://cdn.pixabay.com/video/2021/08/07/60742-573496615_large.mp4",
                preview_image_url="https://cdn.pixabay.com/photo/2017/11/14/05/41/taipei-taiwan-2947814_1280.jpg"
            )
        elif action == "location":
            reply = LocationMessage(
                title="台北101",
                address="台北市信義路五段7號",
                latitude=25.03409570979266,
                longitude=121.564536536767
            )
        else:
            response = ask_gemini(action)
            reply = TextMessage(text=response)

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )

# ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
